In [ ]:
!pip uninstall googletrans
!pip install googletrans==4.0.0-rc1

#### 한 줄씩 추가
- 속도는 느리나 api 한도 초과 후 데이터가 날라가는 것을 방지

In [ ]:
from datasets import load_dataset  # pip install datasets
import googletrans  # pip install googletrans==4.0.0-rc1
import pandas as pd

def BTS(input_str, translator):
    # target language로의 번역 수행
    temp_result = translator.translate(input_str, dest='en').text
    # 다시 source language로의 번역 수행
    result = translator.translate(temp_result, dest='ko').text
    return temp_result, result

# googletrans api 로드
translator = googletrans.Translator()

# 증강할 데이터 로드
dataset = pd.read_csv("train.csv")

# data frame으로 변경
ids, texts, targets, urls, dates = [], [], [], [], []
for index, row in dataset.iterrows():
    ids.append(row['ID'])
    texts.append(row['text'])
    targets.append(row['target'])
    urls.append(row['url'])
    dates.append(row['date'])

df = pd.DataFrame({'ID': ids, 'text': texts,
                   'target': targets, 'url': urls, 'date': dates})

eng_trans_data, augmented_data, augmented_ind, wrong_ind = [], [], [], []
for i in range(len(df)):
    try:
        data = df['text'].iloc[i]
        eng_trans, augmented = BTS(data, translator)
        
        eng_trans_data.append(eng_trans)
        augmented_data.append(augmented)
        augmented_ind.append(i)
        
        # df DataFrame에 결과 반영
        df.at[i, 'eng'] = eng_trans
        df.at[i, 'text'] = augmented
        
        # 한 줄씩 CSV 파일에 추가
        df.iloc[[i]].to_csv('google_augmented_dataset.csv', mode='a', header=False, index=False)
        
    except:
        wrong_ind.append(i)
        
    if i % 200 == 0:
        print(i, " 번째 진행 중...")


#### 한번에 df에 추가

In [ ]:
from datasets import load_dataset  # pip install datasets
import googletrans  # pip install googletrans==4.0.0-rc1
import pandas as pd

def BTS(input_str, translator):
    # target language로의 번역 수행
    temp_result = translator.translate(input_str, dest='en').text
    # 다시 source language로의 번역 수행
    result = translator.translate(temp_result, dest='ko').text
    return temp_result, result

# 사용할 번역기 api 로드
translator = googletrans.Translator()

# 증강할 데이터 로드
# dataset = load_dataset('klue', 'ynat')
dataset = pd.read_csv("/data/ephemeral/data/train_p2g_added.csv")
dataset = dataset[3000:5000]

# data frame으로 변경
ids, texts, targets, urls, dates = [], [], [], [], []
for index, row in dataset.iterrows():
    ids.append(row['ID'])
    texts.append(row['text'])
    targets.append(row['target'])
    urls.append(row['url'])
    dates.append(row['date'])

df = pd.DataFrame({'ID': ids, 'text': texts,
                   'target': targets, 'url': urls, 'date': dates})

eng_trans_data, augmented_data, augmented_ind, wrong_ind = [], [], [], []
for i in range(len(df)):
    try:
        data = df['text'].iloc[i]
        eng_trans, augmented = BTS(data, translator)
        
        eng_trans_data.append(eng_trans)
        augmented_data.append(augmented)#
        augmented_ind.append(i)#
        #df.iloc[augmented_ind]['text'] = augmented
        #df.iloc[augmented_ind]['eng'] = eng
    except:
        wrong_ind.append(i)
    if i%200 == 0 : print(i," 번쨰 진행 중...")

df = df.iloc[augmented_ind]
df['eng'] = eng_trans_data
df['text'] = augmented_data
print(df)
df.to_csv('augmented_dataset.csv', mode='a', index=False)
